## **1 - Importing Libraries & Utilities**



In [1]:
from google.colab import files
import pandas as pd
import numpy as np
from IPython.display import clear_output
from time import sleep, perf_counter
from datetime import timedelta
from google.colab import files
import warnings
warnings.filterwarnings('ignore')

In [2]:
from IPython.display import clear_output
#!pip install selenium
!pip install --upgrade selenium
!apt-get update
!apt-get install chromium chromium-driver
clear_output()

In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import time
# Define the proxy server
PROXY = "IpOfTheProxy:PORT"

def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    #options.add_argument("--proxy-server=%s" % PROXY)
    driver = webdriver.Chrome(options=options)
    #driver.set_page_load_timeout(20)
    return driver
#
driver = web_driver()

## **2 - Canada's Provinces and Territories - Capital Cities**

In [4]:
region_list = ['Newfoundland and Labrador', 'Prince Edward Island', 'Nova Scotia', 'New Brunswick', 'Quebec', 'Ontario', 'Manitoba', 'Saskatchewan', 'Alberta', 'British Columbia', 'Yukon', 'Northwest Territories', 'Nunavut']

In [5]:
capitals_ = {}
capitals_[region_list[0]] = "saint-john/CYSJ"
capitals_[region_list[1]] = "charlottetown/CYYG"
capitals_[region_list[2]] = "halifax/CYHZ"
capitals_[region_list[3]] = "fredericton/CYFC"
capitals_[region_list[4]] = "quebec-city/CYQB"
capitals_[region_list[5]] = "toronto/CYTZ"
capitals_[region_list[6]] = "winnipeg/CYWG"
capitals_[region_list[7]] = "regina/CYQR"
capitals_[region_list[8]] = "edmonton/CYEG"
capitals_[region_list[9]] = "victoria/CYWH"
capitals_[region_list[10]] = "whitehorse/CYXY"
capitals_[region_list[11]] = "yellowknife/CYZF"
capitals_[region_list[12]] = "iqaluit/CYFB"
#capitals_

## **3 - Weather Data Scraping -  WeatherUnderground**

### **3.1 - Function - Scrap Table from WeatherUnderground**

In [6]:
def get_table(place_, date_):
  url_ = 'https://www.wunderground.com/history/monthly/ca/' + place_ +'/date/'+ date_
  driver.get(url_)
  time.sleep(10)
  #
  tables = driver.find_elements(By.TAG_NAME, value = "table")
  list_ = [(date_ +'-' + day) for day in tables[2].text.split('\n')[1:]]
  header_ = ['Time', 'Temp_Max (°F)', 'Temp_Avg (°F)', 'Temp_Min (°F)', 'Hum_Max (%)', 'Hum_Avg (%)', 'Hum_Min (%)', 'Wind_Speed_Max (mph)', 'Wind_Speed_Avg (mph)', 'Wind_Speed_Min (mph)']
  #
  temp = pd.DataFrame(np.array([tables[3].text.split('\n')[1:], tables[5].text.split('\n')[1:], tables[6].text.split('\n')[1:]]).T)
  # Temperature: °F - Speed : mph
  dd = pd.DataFrame(list_)
  for j in range(0, temp.shape[1]):
    dd = pd.concat([dd, temp.iloc[:,j].str.split(' ', expand = True)], axis = 1)
  dd.columns = header_
  return dd

### **3.2 - Get Tables**

In [7]:
import sys
print(sys.getrecursionlimit())

1000


In [8]:
sys.setrecursionlimit(1500)

In [ ]:
start_time = perf_counter()
df = pd.DataFrame()
for region_ in region_list[8:9]:                 #   [8] for Alberta
  place_ = capitals_[region_]
  start_date = '2004-12'
  date_ = start_date
  #
  while date_ < '2024-01':
    print(region_)
    print(date_)#.split('-')[0])
    temp = get_table(place_, date_)
    temp = pd.concat([temp['Time'], pd.DataFrame([region_] * temp.shape[0], columns = ['Region']), temp.iloc[:,1:]], axis = 1)
    date_ = str(np.datetime64(date_) + np.timedelta64(1, 'M'))
    clear_output()
    #
    df = pd.concat([df, temp], axis = 0, ignore_index = True)
    del temp
    #
  df['Time'] = pd.to_datetime(df['Time'])
Duration_ =  (perf_counter() - start_time)/60
print('Duration = ', np.round(Duration_, 1), 'mins' )

In [ ]:
df.to_excel('DSMP_Weather_Data_Alberta.xlsx', index = False, header = True, freeze_panes = (1, 0))
files.download('DSMP_Weather_Data_Alberta.xlsx')